In [2]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset'

In [6]:
#!/usr/bin/env python3
"""
labeling_tool.py – Quick labeling utility for the **Open Thermal Pose** training images.

➔ Purpose
    • Shows every image from a directory one‑by‑one.
    • You press a shortcut key to assign the correct activity label.
    • The <image_path,label> pair is appended to a CSV (it auto‑skips files already labelled).

➔ Usage
    python labeling_tool.py

Controls
    1‑9 0 q w e r t y u i o p a s d f g h j → pick activity
    k                  → skip image
    Esc                → quit

Hints
    • Re‑run the script any time; it resumes where you left off.  
    • Edit the LABELS list or KEY_MAP dictionary below to match your project exactly.
"""

import cv2
import os
import csv
from pathlib import Path

# ➞⃣ Activity labels
LABELS = ["walking","LOOKING_STRAIGHT","STANDING","jumping_climbing", "suspicious_look", "EXERCISE_BODY_SWING", "SITTING_STANDING" ,"fighting","gesturing","lOOKING_UP"    
]

KEY_MAP = {
    ord("1"): 0, ord("2"): 1, ord("3"): 2, ord("4"): 3, ord("5"): 4,
    ord("6"): 5, ord("7"): 6, ord("8"): 7, ord("9"): 8, ord("0"): 9,
}

DATA_DIR = "artifacts/OPEN_THERMAL_IMAGE/val/images"
OUT_CSV = "activity_labels_val.csv"
LABEL_LIST_CSV = "activity_labels.csv"
EXTENSIONS = [".png", ".jpg", ".jpeg"]

def save_label_file(path):
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Index", "Activity"])
        for idx, label in enumerate(LABELS):
            writer.writerow([idx, label])
    print(f"📄 Activity label list saved to {path}")

def load_done(csv_path):
    done = set()
    if os.path.exists(csv_path):
        with open(csv_path, newline="") as f:
            done.update(row[0] for row in csv.reader(f) if row)
    return done

def gather_images(root, exts):
    files = []
    for dirpath, _, filenames in os.walk(root):
        for fname in filenames:
            if Path(fname).suffix.lower() in exts:
                files.append(Path(dirpath) / fname)
    return sorted(files)

def overlay_legend(img):
    out = img.copy()
    x, y = 12, 22
    for keycode, idx in KEY_MAP.items():
        label = LABELS[idx]
        cv2.putText(out, f"{chr(keycode)}: {label}", (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1, cv2.LINE_AA)
        y += 18
    return out

def main():
    images = gather_images(DATA_DIR, [ext.lower() for ext in EXTENSIONS])
    if not images:
        print("❌ No images found – check DATA_DIR and EXTENSIONS.")
        return

    save_label_file(LABEL_LIST_CSV)
    done = load_done(OUT_CSV)
    print(f"🔎 Found {len(images)} files ({len(done)} already labelled).")

    with open(OUT_CSV, "a", newline="") as csvfile:
        writer = csv.writer(csvfile)

        for img_path in images:
            p = str(img_path)
            if p in done:
                continue

            img = cv2.imread(p)
            if img is None:
                print(f"⚠️ Skipping unreadable file: {p}")
                continue

            shown = overlay_legend(img)
            cv2.imshow("Thermal‑Pose Labeler", shown)
            key = cv2.waitKey(0)

            if key == 27:
                break
            if key == ord("k"):
                continue
            if key in KEY_MAP:
                label = LABELS[KEY_MAP[key]]
                writer.writerow([p, label])
                csvfile.flush()
                print(f"✔︎ {p} → {label}")
            else:
                print("❔ Unknown key – use mapped keys or Esc (quit).")

    cv2.destroyAllWindows()
    print("🏋️ Finished.")

if __name__ == "__main__":
    main()


📄 Activity label list saved to activity_labels.csv
🔎 Found 934 files (0 already labelled).
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_1_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_2_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_3_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_4_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_5_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_6_1_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_7_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_8_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_1_9_450_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_2_1_143_100_1.png → LOOKING_STRAIGHT
✔︎ artifacts/OPEN_THERMAL_IMAGE/val/images/131_1_2_2_108_101_1.png → LOOKING_STRAIGHT
✔